# Video Translator Notebook per Google Colab

Questo notebook consente di eseguire il processo di traduzione video con clonazione vocale. (//TODO: Lip Sync)

Assicurati di avere i file necessari (video in input e, se desiderato, campioni vocali in formato WAV) e che la runtime disponga di una GPU per ottenere prestazioni ottimali.

Il codice è stato sviluppato da Mike Gazzaruso ed è rilasciato sotto licenza GNU/GPL v3.

### Per prima cosa, controlliamo se l'istanza è privvista di GPU. Si consiglia di utilizzare un'istanza con GPU perché i calcoli sarebbero estremamente lenti sfruttando solamente la CPU.

In [ ]:
# Controllo della disponibilità di GPU
import torch

if torch.cuda.is_available():
    print("GPU disponibile! Utilizzo della GPU.")
else:
    print("GPU non disponibile. Si consiglia l'utilizzo di una runtime con GPU per prestazioni migliori.")

### Procediamo ora a disinstallare eventuali pacchetti già installati sull'istanza e ad installare le versioni specifiche dei pacchetti che ci interessano.

In [ ]:
# Rimuovi eventuali versioni di sentence-transformers, transformers e tokenizers già installate
!pip uninstall -y sentence-transformers transformers tokenizers

# Aggiorna pip
!pip install --upgrade pip

# Installa le dipendenze principali
!pip install torch torchvision torchaudio --quiet
!pip install librosa moviepy openai-whisper tortoise-tts pydub opencv-python soundfile demucs --quiet

# Installa transformers==4.31.0, che installerà automaticamente una versione compatibile di tokenizers
!pip install transformers==4.31.0 --quiet


## Importazione delle librerie

In questa cella importiamo tutte le librerie utilizzate dallo script.

In [2]:
import os
import subprocess
import torch
import librosa
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip
import whisper
from transformers import pipeline, MBartForConditionalGeneration, MBart50TokenizerFast
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice
import cv2
from pydub import AudioSegment
import tempfile
import shutil
import time
import argparse
import sys
import datetime
import soundfile as sf
import warnings
import traceback
import json
import hashlib
import pickle
import demucs.separate
warnings.filterwarnings('ignore')


## Definizione della classe `VideoTranslator`

La seguente cella contiene la definizione completa della classe `VideoTranslator` con tutti i metodi necessari per:
- Inizializzare i modelli
- Estrarre e trascrivere l'audio
- Tradurre il testo
- Generare segmenti audio con voce clonata
- Combinare i segmenti audio e il video originale

Il codice è ampiamente commentato per una migliore comprensione.

In [3]:
class VideoTranslator:
    def __init__(self, source_lang="it", target_lang="en", voice_samples_dir=None, input_video_path=None, use_cache=True):
        """
        Inizializza il traduttore video.

        Args:
            source_lang: Lingua sorgente (default: Italiano)
            target_lang: Lingua target (default: Inglese)
            voice_samples_dir: Directory contenente campioni vocali per il clonaggio
            input_video_path: Percorso del video in input per creare una directory dedicata
            use_cache: Se usare la cache per campioni e modelli (default: True)
        """
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.voice_samples_dir = voice_samples_dir
        self.use_cache = use_cache

        # Creazione di una directory dedicata per le conversioni
        if input_video_path:
            video_filename = os.path.basename(input_video_path).split('.')[0]
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            work_dir_name = f"{video_filename}_{timestamp}"

            # Nei notebook non esiste __file__, usa os.getcwd()
            script_dir = os.getcwd()
            self.temp_dir = os.path.join(script_dir, "conversions", work_dir_name)
            os.makedirs(self.temp_dir, exist_ok=True)
            print(f"Working directory created: {self.temp_dir}")
        else:
            self.temp_dir = tempfile.mkdtemp()

        # Setup della directory di cache
        script_dir = os.getcwd()
        self.cache_dir = os.path.join(script_dir, "cache")
        os.makedirs(self.cache_dir, exist_ok=True)


        # Debug: controllo della directory dei campioni vocali
        if voice_samples_dir:
            print(f"Voice samples directory specified: {voice_samples_dir}")
            if os.path.exists(voice_samples_dir):
                print(f"Directory exists!")
                wav_files = [f for f in os.listdir(voice_samples_dir) if f.endswith('.wav')]
                print(f"WAV files found: {len(wav_files)}")
                if wav_files:
                    print(f"Examples: {wav_files[:3]}")
            else:
                print(f"Directory does NOT exist!")

        print("Initializing models...")

        # Inizializza il modello di riconoscimento vocale Whisper
        self.transcriber = self._load_whisper_model()

        # Inizializza il modello di traduzione MBart
        self.translator_model, self.translator_tokenizer = self._load_translation_model()

        # Controlla la disponibilità della GPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
        if device.type == 'cpu':
            print("WARNING: No GPU detected. Tortoise-TTS may be very slow on CPU.")

        # Inizializza Tortoise TTS per il clonaggio vocale
        self.tts = TextToSpeech(device=device)

        # Mappa dei codici lingua per MBart
        self.lang_map = {
            "it": "it_IT",  # Italiano
            "en": "en_XX",  # Inglese
            "fr": "fr_XX",  # Francese
            "es": "es_XX",  # Spagnolo
            "de": "de_DE",  # Tedesco
            "zh": "zh_CN",  # Cinese
            "ru": "ru_RU",  # Russo
            "ja": "ja_XX",  # Giapponese
            "pt": "pt_XX",  # Portoghese
            "ar": "ar_AR",  # Arabo
            "hi": "hi_IN"   # Hindi
        }

        # Inizializza le variabili per il condizionamento vocale
        self.voice_samples = None
        self.conditioning_latents = None

        # Carica la voce dalla cache se disponibile
        if self.use_cache and self.voice_samples_dir:
            self._load_cached_voice()

        print("Models loaded successfully.")

    def _load_whisper_model(self):
        """Carica il modello Whisper, utilizzando la cache se disponibile."""
        model_cache_path = os.path.join(self.cache_dir, "whisper_model.pkl")

        if self.use_cache and os.path.exists(model_cache_path):
            try:
                print("Loading Whisper model from cache...")
                with open(model_cache_path, 'rb') as f:
                    model = pickle.load(f)
                return model
            except Exception as e:
                print(f"Error loading cached Whisper model: {e}")
                print("Loading fresh model...")

        model = whisper.load_model("medium")

        # Salva il modello in cache
        if self.use_cache:
            try:
                with open(model_cache_path, 'wb') as f:
                    pickle.dump(model, f)
            except Exception as e:
                print(f"Failed to cache Whisper model: {e}")

        return model

    def _load_translation_model(self):
        """Carica il modello di traduzione MBart, utilizzando la cache se disponibile."""
        model_name = "facebook/mbart-large-50-many-to-many-mmt"

        tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
        model = MBartForConditionalGeneration.from_pretrained(model_name)

        return model, tokenizer

    def _get_voice_cache_path(self):
        """Genera un percorso unico per la cache basato sui campioni vocali."""
        if not self.voice_samples_dir:
            return None

        # Crea un hash del contenuto della directory dei campioni vocali
        hash_obj = hashlib.md5()
        wav_files = sorted([f for f in os.listdir(self.voice_samples_dir) if f.endswith('.wav')])

        for wav_file in wav_files:
            file_path = os.path.join(self.voice_samples_dir, wav_file)
            file_stat = os.stat(file_path)
            hash_obj.update(f"{wav_file}_{file_stat.st_size}_{file_stat.st_mtime}".encode())

        dir_hash = hash_obj.hexdigest()
        return os.path.join(self.cache_dir, f"voice_latents_{dir_hash}.pt")

    def _load_cached_voice(self):
        """Carica i latenti vocali dalla cache se disponibili."""
        cache_path = self._get_voice_cache_path()

        if not cache_path:
            return False

        if os.path.exists(cache_path):
            try:
                print(f"Loading cached voice from {cache_path}")
                cached_data = torch.load(cache_path)
                self.voice_samples = cached_data.get('voice_samples')
                self.conditioning_latents = cached_data.get('conditioning_latents')
                return True
            except Exception as e:
                print(f"Error loading cached voice: {e}")

        return False

    def _save_voice_to_cache(self, voice_samples, conditioning_latents):
        """Salva i latenti vocali nella cache."""
        if not self.use_cache:
            return

        cache_path = self._get_voice_cache_path()
        if not cache_path:
            return

        try:
            print(f"Saving voice to cache: {cache_path}")
            cache_data = {
                'voice_samples': voice_samples,
                'conditioning_latents': conditioning_latents
            }
            torch.save(cache_data, cache_path)
        except Exception as e:
            print(f"Error saving voice to cache: {e}")

    def clear_cache(self, voice_only=False):
        """Pulisce la directory della cache.

        Args:
            voice_only: Se True, pulisce solo la cache vocale
        """
        if voice_only:
            print("Clearing voice cache...")
            voice_cache_files = [f for f in os.listdir(self.cache_dir) if f.startswith("voice_latents_")]
            for file in voice_cache_files:
                try:
                    os.remove(os.path.join(self.cache_dir, file))
                except Exception as e:
                    print(f"Error removing {file}: {e}")
        else:
            print("Clearing all cache...")
            for file in os.listdir(self.cache_dir):
                try:
                    file_path = os.path.join(self.cache_dir, file)
                    if os.path.isfile(file_path):
                        os.remove(file_path)
                except Exception as e:
                    print(f"Error removing {file}: {e}")

    def extract_audio(self, video_path):
        """Estrae l'audio dal video."""
        print("Extracting audio from video...")
        audio_path = os.path.join(self.temp_dir, "extracted_audio.wav")
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_path, codec='pcm_s16le', fps=16000)
        return audio_path

    def transcribe_audio(self, audio_path):
        """Trascrive l'audio utilizzando Whisper."""
        print("Transcribing audio...")
        result = self.transcriber.transcribe(audio_path, language=self.source_lang)
        return result

    def translate_text(self, text):
        """Traduce il testo nella lingua target utilizzando MBart."""
        print("Translating text...")
        self.translator_tokenizer.src_lang = self.lang_map[self.source_lang]

        # Tokenizza il testo
        encoded = self.translator_tokenizer(text, return_tensors="pt")

        # Genera la traduzione
        generated_tokens = self.translator_model.generate(
            **encoded,
            forced_bos_token_id=self.translator_tokenizer.lang_code_to_id[self.lang_map[self.target_lang]]
        )

        # Decodifica la traduzione
        translation = self.translator_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation

    def align_segments(self, transcription, translation):
        """Allinea i segmenti tradotti con i tempi originali."""
        print("Aligning segments...")
        aligned_segments = []
        segments = transcription["segments"]

        # Suddivide la traduzione in segmenti proporzionali agli originali
        for segment in segments:
            original_text = segment["text"]
            start_time = segment["start"]
            end_time = segment["end"]

            # Ottiene la traduzione del segmento
            translation_segment = self.translate_text(original_text)

            aligned_segments.append({
                "start": start_time,
                "end": end_time,
                "text": translation_segment
            })

        return aligned_segments

    def prepare_voice_samples(self):
        """Prepara i campioni vocali per il clonaggio."""
        if self.conditioning_latents is not None and self.voice_samples is not None:
            print("Using cached voice conditioning latents")
            return self.voice_samples, self.conditioning_latents

        if not self.voice_samples_dir or not os.path.exists(self.voice_samples_dir):
            print("Voice samples directory not found.")
            return None, None

        wav_files = [f for f in os.listdir(self.voice_samples_dir) if f.endswith('.wav')]
        if not wav_files:
            print("No WAV files found in voice samples directory.")
            return None, None

        print(f"Loading {len(wav_files)} voice samples...")

        # Carica i campioni vocali
        voice_samples = []
        for wav_file in wav_files:
            file_path = os.path.join(self.voice_samples_dir, wav_file)
            try:
                # Carica audio a 24kHz (usato da Tortoise)
                audio, sr = librosa.load(file_path, sr=24000, mono=True)
                # Converte in tensor e aggiunge dimensione batch
                audio_tensor = torch.FloatTensor(audio).unsqueeze(0).to(self.tts.device)
                voice_samples.append(audio_tensor)
            except Exception as e:
                print(f"Error loading file {wav_file}: {e}")

        if not voice_samples:
            print("No valid voice samples loaded.")
            return None, None

        # Genera i latenti per il condizionamento una sola volta
        print("Generating voice conditioning latents...")
        try:
            conditioning_latents = self.tts.get_conditioning_latents(voice_samples)

            # Salva in cache per usi futuri
            self._save_voice_to_cache(voice_samples, conditioning_latents)

            # Memorizza nell'istanza per riutilizzo
            self.voice_samples = voice_samples
            self.conditioning_latents = conditioning_latents

            return voice_samples, conditioning_latents
        except Exception as e:
            print(f"Error generating voice conditioning latents: {e}")
            print(traceback.format_exc())
            return None, None

    def clone_voice(self, text, segment_idx=0):
        """Genera audio con voce clonata per il segmento specificato."""
        print(f"Generating audio for segment {segment_idx + 1}...")

        try:
            # Prepara il condizionamento vocale se non già fatto
            if self.conditioning_latents is None:
                voice_samples, conditioning_latents = self.prepare_voice_samples()
            else:
                voice_samples, conditioning_latents = self.voice_samples, self.conditioning_latents

            if conditioning_latents is not None:
                print(f"Using cached voice conditioning for generation...")
                # Utilizza i latenti di condizionamento in cache
                gen = self.tts.tts_with_preset(text,
                                              conditioning_latents=conditioning_latents,
                                              preset="fast")  # Options: ultra_fast, fast, standard, high_quality
            else:
                print("Using default voice...")
                gen = self.tts.tts(text, voice_samples=None)

            # Ottiene il primo risultato
            audio = gen[0].cpu().numpy()
            return audio
        except Exception as e:
            print(f"Error during audio generation: {e}")
            print(traceback.format_exc())

            # Crea audio silenzioso come fallback
            print("Creating silent audio as fallback...")
            return np.zeros(int(24000 * 3))  # 3 secondi di silenzio a 24kHz

    def generate_audio_segments(self, aligned_segments):
        """Genera segmenti audio per ogni segmento tradotto."""
        print("Generating audio segments...")
        audio_segments = []

        for i, segment in enumerate(aligned_segments):
            text = segment["text"]
            start_time = segment["start"]
            end_time = segment["end"]
            duration = end_time - start_time

            # Genera audio con voce clonata
            audio = self.clone_voice(text, i)

            # Salva l'audio temporaneamente con numpy.save
            temp_audio_path = os.path.join(self.temp_dir, f"segment_{i}.npy")
            np.save(temp_audio_path, audio)

            # Converte l'array numpy in WAV usando ffmpeg
            wav_path = os.path.join(self.temp_dir, f"segment_{i}.wav")
            temp_raw_path = os.path.join(self.temp_dir, f"segment_{i}.raw")

            try:
                # Salva prima come dati PCM raw
                with open(temp_raw_path, 'wb') as f:
                    # Converte in 16-bit PCM
                    audio_16bit = (audio * 32767).astype(np.int16)
                    audio_16bit.tofile(f)

                # Usa ffmpeg per convertire in WAV
                result = subprocess.call([
                    "ffmpeg", "-y",
                    "-f", "s16le",  # 16-bit signed little endian
                    "-ar", "24000",  # frequenza 24kHz
                    "-ac", "1",      # 1 canale (mono)
                    "-i", temp_raw_path,
                    wav_path
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

                if result != 0 or not os.path.exists(wav_path):
                    print(f"ERROR: Failed to create WAV file for segment {i}")
                    continue

                # Carica l'audio per regolare la velocità
                audio_segment = AudioSegment.from_file(wav_path)

                # Calcola il fattore di velocità per il sincronismo labiale
                current_duration = len(audio_segment) / 1000.0  # durata in secondi
                speed_factor = current_duration / duration if duration > 0 else 1.0

                # Regola la velocità se necessario
                if abs(speed_factor - 1.0) > 0.1:  # Differenza significativa
                    if speed_factor > 1.5:  # Limita l'accelerazione
                        speed_factor = 1.5

                    # Usa ffmpeg per regolare la velocità mantenendo il pitch
                    adjusted_path = os.path.join(self.temp_dir, f"adjusted_{i}.wav")
                    result = subprocess.call([
                        "ffmpeg", "-y", "-i", wav_path,
                        "-filter:a", f"atempo={speed_factor}",
                        adjusted_path
                    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

                    if result != 0 or not os.path.exists(adjusted_path):
                        print(f"ERROR: Failed to adjust audio speed for segment {i}")
                        adjusted_path = wav_path
                    else:
                        audio_segment = AudioSegment.from_file(adjusted_path)
                else:
                    adjusted_path = wav_path

                try:
                    if os.path.exists(temp_raw_path):
                        os.remove(temp_raw_path)
                except:
                    pass

                audio_segments.append({
                    "path": adjusted_path,
                    "start": start_time,
                    "end": end_time,
                    "duration": len(audio_segment) / 1000.0
                })
            except Exception as e:
                print(f"Error generating audio segment {i}: {e}")
                print(traceback.format_exc())

        return audio_segments

    def combine_audio_segments(self, audio_segments, original_audio_path):
        """Combina i segmenti audio utilizzando la separazione vocale AI."""
        print("Combining audio segments with AI voice separation...")

        if not audio_segments:
            print("WARNING: No audio segments to combine.")
            return original_audio_path

        # Step 1: Usa Demucs per separare voce e musica/fondo nell'audio originale
        print("Separating voice and background from original audio...")

        try:
            demucs_output_dir = os.path.join(self.temp_dir, "demucs_output")
            os.makedirs(demucs_output_dir, exist_ok=True)

            wav_original = os.path.join(self.temp_dir, "original_for_separation.wav")

            if not original_audio_path.lower().endswith('.wav'):
                subprocess.call([
                    "ffmpeg", "-y", "-i", original_audio_path, wav_original
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            else:
                shutil.copy(original_audio_path, wav_original)

            print("Running Demucs AI separation...")

            demucs.separate.main(["--out", demucs_output_dir, "--two-stems", "vocals", wav_original])

            htdemucs_dir = os.path.join(demucs_output_dir, "htdemucs")
            if not os.path.exists(htdemucs_dir):
                possible_dirs = [d for d in os.listdir(demucs_output_dir) if os.path.isdir(os.path.join(demucs_output_dir, d))]
                if possible_dirs:
                    htdemucs_dir = os.path.join(demucs_output_dir, possible_dirs[0])
                else:
                    raise FileNotFoundError("Could not find Demucs output directory")

            base_name = os.path.basename(wav_original).split('.')[0]
            no_vocals_path = os.path.join(htdemucs_dir, base_name, "no_vocals.wav")
            vocals_path = os.path.join(htdemucs_dir, base_name, "vocals.wav")

            if not (os.path.exists(no_vocals_path) and os.path.exists(vocals_path)):
                raise FileNotFoundError(f"Demucs output files not found. Looking for: {no_vocals_path} and {vocals_path}")

            print("Successfully separated voice and background!")

            background_audio = AudioSegment.from_file(no_vocals_path)
            final_audio = AudioSegment.silent(duration=len(background_audio))

            for segment in audio_segments:
                try:
                    path = segment["path"]
                    if os.path.exists(path):
                        start_ms = int(segment["start"] * 1000)
                        audio = AudioSegment.from_file(path)
                        final_audio = final_audio.overlay(audio, position=start_ms)
                    else:
                        print(f"WARNING: Missing audio file: {path}")
                except Exception as e:
                    print(f"Error during audio segment combination: {e}")

            final_mixed_audio = background_audio.overlay(final_audio)

            final_audio_path = os.path.join(self.temp_dir, "final_audio.wav")
            final_mixed_audio.export(final_audio_path, format="wav")

            return final_audio_path

        except Exception as e:
            print(f"Error in AI voice separation: {e}")
            print(traceback.format_exc())

            print("Falling back to basic audio combination...")

            original_audio = AudioSegment.from_file(original_audio_path)
            final_audio = AudioSegment.silent(duration=len(original_audio))

            for segment in audio_segments:
                try:
                    path = segment["path"]
                    if os.path.exists(path):
                        start_ms = int(segment["start"] * 1000)
                        audio = AudioSegment.from_file(path)
                        final_audio = final_audio.overlay(audio, position=start_ms)
                except Exception as e:
                    print(f"Error: {e}")

            final_audio_path = os.path.join(self.temp_dir, "final_audio.wav")
            final_audio.export(final_audio_path, format="wav")

            return final_audio_path

    def combine_video_and_audio(self, video_path, audio_path, output_path):
        """Combina il video originale con l'audio tradotto."""
        print("Combining video and audio...")
        command = [
            "ffmpeg", "-y",
            "-i", video_path,
            "-i", audio_path,
            "-c:v", "copy",
            "-c:a", "aac",
            "-map", "0:v:0",
            "-map", "1:a:0",
            output_path
        ]
        subprocess.call(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return output_path

    def cleanup(self):
        """Pulisce i file temporanei."""
        print("Cleaning up temporary files...")
        # In questa versione manteniamo i file nella directory dedicata
        # Non eliminiamo self.temp_dir
        pass

    def process_video(self, video_path, output_path):
        """Processa il video dall'inizio alla fine."""
        try:
            start_time = time.time()
            print(f"Starting video processing: {video_path}")

            # Prepara il condizionamento vocale se applicabile
            if self.voice_samples_dir:
                self.prepare_voice_samples()

            # Estrai l'audio dal video
            audio_path = self.extract_audio(video_path)

            # Trascrivi l'audio
            transcription = self.transcribe_audio(audio_path)

            # Salva la trascrizione per riferimento
            transcription_path = os.path.join(self.temp_dir, "transcription.json")
            with open(transcription_path, 'w', encoding='utf-8') as f:
                json.dump(transcription, f, ensure_ascii=False, indent=2)

            # Allinea i segmenti tradotti
            aligned_segments = self.align_segments(transcription, None)

            # Salva i segmenti allineati per riferimento
            aligned_path = os.path.join(self.temp_dir, "aligned_segments.json")
            with open(aligned_path, 'w', encoding='utf-8') as f:
                json.dump(aligned_segments, f, ensure_ascii=False, indent=2)

            # Genera i segmenti audio con voce clonata
            audio_segments = self.generate_audio_segments(aligned_segments)

            # Combina i segmenti audio
            final_audio_path = self.combine_audio_segments(audio_segments, audio_path)

            # Combina il video originale con l'audio tradotto
            result_path = self.combine_video_and_audio(video_path, final_audio_path, output_path)

            elapsed_time = time.time() - start_time
            print(f"Processing completed in {elapsed_time:.2f} seconds")
            print(f"Translated video saved to: {output_path}")

            return result_path
        except Exception as e:
            print(f"Error during video processing: {e}")
            print(traceback.format_exc())
            return None
        finally:
            self.cleanup()

def main():
    parser = argparse.ArgumentParser(description='Video Translator with voice cloning and lip sync')
    parser.add_argument('--input', required=True, help='Path to input video')
    parser.add_argument('--output', required=True, help='Path to output video')
    parser.add_argument('--source-lang', default='it', help='Source language (default: it)')
    parser.add_argument('--target-lang', default='en', help='Target language (default: en)')
    parser.add_argument('--voice-samples', help='Directory containing voice samples for voice cloning')
    parser.add_argument('--no-cache', action='store_true', help='Disable caching of voice samples and models')
    parser.add_argument('--clear-cache', action='store_true', help='Clear all cached data before processing')
    parser.add_argument('--clear-voice-cache', action='store_true', help='Clear only voice cache before processing')
    parser.add_argument('--keep-temp', action='store_true', help='Keep temporary files after processing')

    args = parser.parse_args()

    # Verifica che il file di input esista
    if not os.path.exists(args.input):
        print(f"Error: Input file {args.input} does not exist.")
        sys.exit(1)

    # Crea la directory 'conversions' nella cartella dello script
    script_dir = os.path.dirname(os.path.abspath(__file__))
    conversions_dir = os.path.join(script_dir, "conversions")
    os.makedirs(conversions_dir, exist_ok=True)

    # Inizializza il video translator
    translator = VideoTranslator(
        source_lang=args.source_lang,
        target_lang=args.target_lang,
        voice_samples_dir=args.voice_samples,
        input_video_path=args.input,
        use_cache=not args.no_cache
    )

    if args.clear_cache:
        translator.clear_cache(voice_only=False)
    elif args.clear_voice_cache:
        translator.clear_cache(voice_only=True)

    translator.process_video(args.input, args.output)



## Utilizzo Interattivo

qui definiamo direttamente i percorsi e utilizziamo il metodo `process_video` in modo interattivo.

Assicurati di caricare il video di input (ad es. `input_video.mp4`) e, se desiderato, la directory dei campioni vocali (`voice_samples`) nell'ambiente Colab.

## Nota
Il processo potrebbe richiedere del tempo, in base alla lunghezza del video e alle risorse disponibili.

Al termine, il video tradotto verrà salvato nel percorso specificato in output_video_path e visualizzato in un player qui sotto.

In [ ]:
import io
import base64
from IPython.display import HTML

# Modifica i seguenti percorsi secondo le tue esigenze
input_video_path = '/content/data/input_video.mov'  # Carica il tuo video in Colab, tasto destro sulla file e "mostra percorso" per copiare il percorso esatto
output_video_path = '/content/data/output_video.mov'
voice_samples_directory = '/content/data/voice_samples'  # La directory deve contenere file .wav per la clonazione vocale

# Inizializza il traduttore video
translator = VideoTranslator(
    source_lang='it',
    target_lang='en',
    voice_samples_dir=voice_samples_directory,
    input_video_path=input_video_path,
    use_cache=True
)

# Processa il video e genera il video tradotto
translator.process_video(input_video_path, output_video_path)

print(f"Video tradotto salvato in: {output_video_path}")

# Leggi il file e codificalo in base64
def get_video_html(video_path):
    with open(output_video_path, "rb") as video_file:
        video_base64 = base64.b64encode(video_file.read()).decode()
    return f"""
    <video width="640" height="360" controls>
        <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """

# Mostra il video direttamente nel notebook
HTML(get_video_html(output_video_path))